In [1]:
import numpy as np
import random
import math 
import copy

# Clases

## Nodos [N]

In [2]:
class Node:
  # propiedades de los nodos [N]
  def __init__(self, tag, x = 0, y = 0, deg = 0):
    self.tag = tag    # etiqueta del nodo
    self.x = x        # índice x
    self.y = y        # índice y
    self.deg = deg    # grado del nodo

  # funciones
  def node_tag(self):
    return self.tag
  def node_deg(self):
    return self.deg
  def node_add_deg(self):
    self.deg = self.deg + 1 

## Aristas [E = (n1, n2)]

In [3]:
class Edge:
  # propiedades de las aristas [E = (n1, n2)]
  def __init__(self, start_node, end_node, weight = 0):
    self.start = start_node   # n1
    self.end = end_node       # n2
    self.weight = weight      # peso de la arista
  
  # funciones
  def edge_n1(self):
    return self.start
  def edge_n2(self):
    return self.end
  def edge_nodes(self):
    return [self.start, self.end]

## Grafos [G(N,E)]

In [4]:
class Graph:
  # propiedades de los grafos [G(N,E)]
  def __init__(self, name, graph_type = 'graph', path = False):
    self.name = name          # nombre del grafo
    self.gtype = graph_type   # tipo del grafo (dirigido o no)
    self.path = path          # camino

    self.nodes = []           # lista de nodos en G
    self.edges = []           # lista de aristas de G
    self.node_lst = []        # lista de nodos en camino
    self.node_weight = []     # lista de pesos de nodos en camino
    self.edge_lst = []        # lista de aristas  en camino

  # funciones
  def add_nodes(self, new_nodes):
    self.nodes.extend(new_nodes)
  def get_nodes(self):
    return self.nodes
  def add_edges(self, new_edges):
    self.edges.extend(new_edges)
  def get_edges(self):
    return self.edges

  ##### función para exportar el grafo #####

  # formato GraphViz (.gv)
  def save_graph(self, graph_name, rand_weight = False):
    
    # generando cadena de texto a guardar
    inside_file = self.gtype + " " + self.name + " {\n"  # nombre y tipo del grafo

    ## agregando nodos ##
    # en camino  
    if self.path:
      for i in range(len(self.nodes)):
        if i in self.node_lst:
          pos_node = self.node_lst.index(i)
          # etiquetado con peso
          inside_file += str(self.nodes[i].tag)+" [label="+str(self.nodes[i].tag)+"("+str(self.node_weight[pos_node])+")];\n"
        # no conectado  
        else: 
          inside_file += str(self.nodes[i].tag)+";\n"
    # sin camino
    else:
      for i in range(len(self.nodes)):
        inside_file += str(self.nodes[i].tag)+";\n"
    
    
    # unión arista
    if self.gtype == 'graph':
      edge_style = "--"
    elif self.gtype == 'digraph':
      edge_style = '->'


    ## agregando aristas ##
    # con peso aleatorio
    if rand_weight:
      for i in range(len(self.edges)):
        inside_file += str(self.edges[i].start) + edge_style + str(self.edges[i].end) + "[w = " +str(round(random.random()*3)+1) + "\"]"+";\n"
    # con peso en camino
    elif self.path:
      for i in range(len(self.edges)):
        inside_file += str(self.edges[i].start) + edge_style + str(self.edges[i].end) + "[w = " +str(self.edges[i].weight) + "\"]"+";\n"
    else: 
      for j in range(len(self.edges)):
        inside_file += str(self.edges[j].start) + edge_style + str(self.edges[j].end) + ";\n"
    # con camino
    if self.path:
      for i in range(len(self.edge_lst)):
        inside_file += str(self.edge_lst[i][0]) + edge_style + str(self.edge_lst[i][1]) +" [color = pink,] ;\n"
 
    # final del archivo
    inside_file += "}"

    # escribiendo archivo
    with open(graph_name + ".gv", 'w') as outfile:
      outfile.write(inside_file)

# Modelos de generación de grafos

## Modelo de malla

In [5]:
def Grid(n, m, name, graph_type = 'graph'):
  # listas iniciales
  nodes = []
  edges = []
    
  # generando nodos
  for j in range(m):
    for i in range(n): 
      nodes.append(Node(i, i, j))
   
  # generando aristas # orilla
  for j in range(m-1):
    edges.append(Edge(j*n + n -1, j*n + 2*n -1))
    for i in range(n-1):
      edges.append(Edge(j*n + i, j*n + i + 1)) 
      edges.append(Edge(j*n + i, j*n + i + n))
  for i in range(n-1): # orilla
    edges.append(Edge(n*(m-1) + i, n*(m-1) + i + 1))

  # generando grafo
  new_graph = Graph(name)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)

  return new_graph

In [6]:
#G.append(Grid(3, 10, "Malla30"))
#G.append(Grid(10, 10, "Malla100"))
#G.append(Grid(25, 20, "Malla500"))

## Modelo de Erdös y Rényi

In [7]:
def Erdos_Renyi(n, m, name, graph_type = 'graph', auto=False):
  # iniciales
  matrix = np.zeros((n, n), dtype=int)    
  nodes = []
  edges = []

  # generando n nodos
  for i in range(n):
    nodes.append(Node(i))

  # generando m aristas
  for i in range(m):  
    while True:
      # aristas entre nodos aleatorios    
      start_node = random.randint(0, n - 1)
      end_node = random.randint(0, n - 1)
      if ( (start_node != end_node) or auto ) and matrix[start_node][end_node] == 0:
       break
    matrix[start_node][end_node] = 1
    matrix[end_node][start_node] = 1

    edges.append(Edge(start_node, end_node))
    edges.append(Edge(end_node, start_node))

  # generando grafo            
  new_graph = Graph(name)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)
  
  return new_graph

In [8]:
#G.append(Erdos_Renyi(30, 50, "ErdRen30_50"))
#G.append(Erdos_Renyi(100, 300, "ErdRen100_300"))
#G.append(Erdos_Renyi(500, 1000, "ErdRen500_1000"))

## Modelo de Gilbert

In [9]:
def Gilbert(n, p, name, gtype = 'graph', auto = False):
  # listas iniciales
  nodes = []
  edges = [] 
   
  # generando vértices
  for i in range(n):
    nodes.append(Node(i))

  # generando aristas
  lim = n + auto - 1
  for i in range(n):
    lim=i+auto
    for j in range(lim):
      if random.random() <= p:
        if auto or (not i == j):
          edges.append(Edge(i, j))

  # generando grafo
  new_graph = Graph(name)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)

  return new_graph

In [10]:
#G.append(Gilbert(30, .3, "Gil30_.3"))
#G.append(Gilbert(100, .2, "Gil100_.2"))
#G.append(Gilbert(500, .1, "Gil500_.1"))

## Modelo geográfico simple

In [11]:
# distancia euclideana (entre nodos)
def euclid_dist(x1, y1, x2, y2):
  dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
  return dist

def Simple_Geo(n, r, name, graph_type = 'graph', auto = False):   
  # listas iniciales
  nodes = []
  edges = [] 

  x_coords=[]
  y_coords=[]
  
  # generando nodos
  for i in range (n):
    x = random.random()
    x_coords.append(x)
    y = random.random()
    y_coords.append(y)
    nodes.append(Node(i, x = x, y = y))
  
  # generando aristas
  for i in range(n):
    for j in range(n - i - 1 + auto):
      distance = euclid_dist(x_coords[i], y_coords[i], x_coords[n-j-1], y_coords[n-j-1])
      if distance < r:
        edges.append(Edge(i, n-j-1, weight = distance))

  # generando grafo
  new_graph = Graph(name)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)

  return new_graph

In [12]:
#G.append(Simple_Geo(30, .2, "Geo30_.2"))
#G.append(Simple_Geo(100, .1, "Geo100_.1"))
#G.append(Simple_Geo(500, .05, "Geo500_.05"))

## Modelo Barabasi-Albert

In [13]:
def Barabasi_Albert(n, d, name, graph_type = 'graph', auto = False):    
  # listas iniciales
  nodes = []
  edges = [] 
  node_list = []

  # generando nodos
  for i in range(n): 
    nodes.append(Node(i))
    node_list.append(0)
    # comparando con nodos anteriores  
    for j in range (0,i): 
      p = 1 - node_list[j] / d
      # generando aristas
      if random.random() < p:
        edges.append(Edge(i, j))
        node_list[i] += 1
        node_list[j] += 1
        nodes[i].node_add_deg()
        nodes[j].node_add_deg()
        
  # generando grafo
  new_graph = Graph(name)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)

  return new_graph

## Modelo Dorogovtsev-Mendes

In [14]:
def Dorogovtsev_Mendes(n, name, graph_type = 'graph', limit = 0):
  # listas iniciales
  nodes = []
  edges = [] 
    
  #Primeros tres nodos  
  for i in range(3):
    nodes.append(Node(i))
  edges.append(Edge(0,1))
  edges.append(Edge(1,2))
  edges.append(Edge(2,0))

  if limit:
    nodes[0].node_add_deg()
    nodes[1].node_add_deg()
    nodes[2].node_add_deg()
    nodes[0].node_add_deg()
    nodes[1].node_add_deg()
    nodes[2].node_add_deg()

  for i in range(n - 3):
    nodes.append(Node(i + 3))
    if limit:
      flag = True
      while flag:
        pick = random.randint(0, len(edges)-1)
        if nodes[edges[pick].edge_n1()].node_deg() < limit and nodes[edges[pick].edge_n2()].node_deg() < limit:
          flag = False
    else:
      pick = random.randint(0,len(edges)-1)
    
    edges.append(Edge(i+3,edges[pick].edge_n1()))
    edges.append(Edge(i+3,edges[pick].edge_n2()))
    if limit:
      nodes[-1].node_add_deg()
      nodes[-1].node_add_deg()
      nodes[edges[pick].edge_n1()].node_add_deg()
      nodes[edges[pick].edge_n2()].node_add_deg()

  # generando grafo
  new_graph = Graph(name)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)

  return new_graph

In [15]:
#G.append(Dor_Men(30, "DorMen30"))
#G.append(Dor_Men(100, "DorMen100"))
#G.append(Dor_Men(500, "DorMen500"))

# Algoritmo de Dijkstra

In [16]:
def rand_weight(a, max_w):    
    grf = copy.deepcopy(a)
    for i in range(len(grf.edges)):
        grf.edges[i].weight = int(round(random.random()*(max_w - 1)) + 1)
    return grf

In [17]:
def Dijkstra(graph):
  g = copy.deepcopy(graph)
    
  new_nodes = len(g.nodes)
  start = round(random.random() * (new_nodes-1))
  end = round(random.random() * (new_nodes-1))

  # árbol de expansión mínima

  matrix = np.zeros((new_nodes, new_nodes), dtype = int)
  for i in range(len(graph.edges)):
    matrix[graph.edges[i].start][graph.edges[i].end] = graph.edges[i].weight
    matrix[graph.edges[i].end][graph.edges[i].start] = graph.edges[i].weight

  # condiciones iniciales
  visited_nodes = [start]
  visited_nodes_w = [0]
  visited_edges = [] 
  adjacent_nodes = np.asarray(np.where(matrix[start] > 0)[0])
  adjacent_edges = []

  # visitando vecinos  
  for i in range(len(adjacent_nodes)):
    adjacent_edges.append([start, adjacent_nodes[i]])
  
  adjacent_edges_w = matrix[start][adjacent_nodes]
  adjacent_nodes_w = matrix[start][adjacent_nodes]    
  
  # sin peso de aristas recorridas
  for i in range(new_nodes):
    matrix[start][i] = 0
    matrix[i][start] = 0 
  # nodos sin explorar     
  while len(adjacent_nodes): 
    pick = np.where(adjacent_nodes_w == min(adjacent_nodes_w))[0]
    pick = pick[0]
    # recorridos
    visited_edges.append(adjacent_edges[pick])
    visited_nodes.append(adjacent_nodes[pick])
    visited_nodes_w.append(adjacent_nodes_w[pick])    
    
    # actualizando  vecinos, nodos, aristas, y pesos
    node_pick = adjacent_nodes[pick]
    new_nodes_lst = np.where(matrix[node_pick] > 0)[0]
    new_edges_lst = []
    
    for i in range(len(new_nodes_lst)):
      new_edges_lst.append([node_pick, new_nodes_lst[i]])
    
    new_edges_W_lst = matrix[node_pick][new_nodes_lst]
    new_nodes_W_lst = new_edges_W_lst + adjacent_nodes_w[pick]

    # actualizando con peso mínimo    
    for i in range(len(new_nodes_lst)):
      # nodos nuevos
      if new_nodes_lst[i] not in adjacent_nodes: 
        adjacent_nodes = np.append(adjacent_nodes, new_nodes_lst[i])
        adjacent_edges.append(new_edges_lst[i])
        adjacent_edges_w = np.append(adjacent_edges_w,new_edges_W_lst[i])
        adjacent_nodes_w = np.append(adjacent_nodes_w,new_nodes_W_lst[i])
      # nodos previos
      else:       
        node_choice = np.where(adjacent_nodes == new_nodes_lst[i])[0] 
        node_choice = node_choice[0]
        # peso del camino viejo vs nuevo
        if adjacent_nodes_w[node_choice] > new_nodes_W_lst[i]: 
          adjacent_edges[node_choice] = new_edges_lst[i]
          adjacent_nodes_w[node_choice] = new_nodes_W_lst[i]
          adjacent_edges_w[node_choice] = new_edges_W_lst[i]
    # quitando el anterior
    for i in range(new_nodes):
      matrix[node_pick][i] = 0
      matrix[i][node_pick] = 0

    adjacent_nodes = np.delete(adjacent_nodes, pick)
    adjacent_nodes_w = np.delete(adjacent_nodes_w, pick)
    adjacent_edges.pop(pick)
    adjacent_edges_w = np.delete(adjacent_edges_w, pick)    

  # grafo con camino
  g.name = g.name + "_" + str(start)
  g.path = True
  g.edge_lst = visited_edges
  g.node_lst = visited_nodes
  g.node_weight = visited_nodes_w
  
  return g

In [18]:
G=[]

G.append(Grid(5,10,"Malla50"))
G.append(Grid(25,40,"Malla1000"))

G.append(Erdos_Renyi(50,100,"ErdRen50")) 
G.append(Erdos_Renyi(1000,2000,"ErdRen1000")) 

G.append(Gilbert(50,.2,"Gil50"))
G.append(Gilbert(1000,.05,"Gil1000"))

G.append(Simple_Geo(50,.3,"Geo50"))
G.append(Simple_Geo(1000,.1,"Geo1000"))

G.append(Barabasi_Albert(50,3,"BarAlb50"))
G.append(Barabasi_Albert(1000,5,"BarAlb1000"))

G.append(Dorogovtsev_Mendes(50,"DorMen50"))
G.append(Dorogovtsev_Mendes(1000,"DorMen1000"))

for i in range(len(G)):
  G[i]=rand_weight(G[i],100)
  G[i].save_graph(G[i].name)
  a=Dijkstra(G[i])
  a.save_graph(a.name)